In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import PIL.Image as Image
from tensorflow.keras import layers
from sklearn.model_selection import KFold
import os
import progressbar
from time import sleep
import zipfile
import cv2
from google.colab import drive
drive.mount('/content/drive')

print("success")
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
success
/content


In [0]:
dim = (224, 224)
data_path = "./dataset/Images/"
def load_image(name) :
    img = Image.open(name).resize(dim)
    img = np.array(img)/255.0
    return img

In [0]:
with zipfile.ZipFile("./drive/My Drive/Images.zip", 'r') as zip_ref:
    zip_ref.extractall("./dataset/")
#labels_path = "./dataset/labels.txt"

In [0]:
open('./drive/My Drive/nparray_images.txt', 'w').close() #wipes file if not empty and creates
open('./drive/My Drive/nparray_labels.txt', 'w').close()

In [0]:
file_images=open('./drive/My Drive/nparray_images.txt','ab')
file_labels=open('./drive/My Drive/nparray_labels.txt','ab')
bar = progressbar.ProgressBar(maxval=20580, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
i=1
for root, dirs, files in os.walk(data_path, topdown=False):
    for name in files:
        image=load_image(os.path.join(root, name))
        if(image.size!=150528):
            print(root,name,"has bad dimensions:",image.shape)
        else:
            label=np.array([root[17:]])
            np.savetxt(file_images,image.reshape((150528)),fmt="%1.4g")
            np.savetxt(file_labels,label,fmt="%s")
        bar.update(i)
        i=i+1
        if(i>=3):
          break
    if(i>=3):
      break
bar.finish()
file_images.close()
file_labels.close()

[========================================================================] 100%


In [0]:
npimages=np.loadtxt('./drive/My Drive/nparray_images.txt')#.reshape((20580,224,224,3))
nplabels=np.loadtxt('./drive/My Drive/nparray_labels.txt')

In [0]:
kf = KFold(n_splits=10, random_state=None, shuffle=True)
kf.get_n_splits(npimages)
X = npimages
y = nplabels
mobilenet = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
                            input_shape=(224,224,3))
mobilenet.trainable = False
model = tf.keras.Sequential([mobilenet, layers.Dense(120, activation='softmax')])
model.summary()

644


In [0]:
print("starting cross val for 10 folds")
i = 1
for train_index, test_index in kf.split(X):
    trainData = X[train_index]
    testData = X[test_index]
    trainLabels = y[train_index]
    testLabels = y[test_index]
    print("\n********* K Fold Cross Val #",i)
    
    model = tf.keras.Sequential([mobilenet, layers.Dense(120, activation='softmax')])
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    num_batches = np.ceil(training_images.samples/training_images.batch_size)
    _=model.fit(trainData,trainLabels,batch_size=64,epochs=10,validation_data=(testData,testLabels))
    i=i+1